<a href="https://colab.research.google.com/github/parkmina365/deep_learning_with_python/blob/main/%EC%BC%80%EB%9D%BC%EC%8A%A4_%EC%B0%BD%EC%8B%9C%EC%9E%90%EC%97%90%EA%B2%8C_%EB%B0%B0%EC%9A%B0%EB%8A%94_%EB%94%A5%EB%9F%AC%EB%8B%9D_5%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5장. 컴퓨터 비전을 위한 딥러닝

## 5.1 합성곱 신경망 소개

In [ ]:
from IPython.display import Image
Image('/content/drive/MyDrive/cakd3_colab/dl_keras/참고자료/cnn.jpg')

In [ ]:
Image('/content/drive/MyDrive/cakd3_colab/dl_keras/참고자료/cnn2.jpg')

In [ ]:
Image('/content/drive/MyDrive/cakd3_colab/dl_keras/참고자료/cnn3.jpg')

In [ ]:
Image('/content/drive/MyDrive/cakd3_colab/dl_keras/참고자료/cnn5.jpg')

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))

In [ ]:
model.summary()
# 26: input 28에서 필터 3을 빼고 1을 더한 값

# (28,28,1) 크기의 특성 맵을 입력으로 받아 (26,26,32) 크기의 특성 맵을 출력
# 입력에 대해 32개의 필터를 적용하고 32개의 출력 채널 각각은 26X26 크기의 배열 값을 가진다.
# 특성 맵의 출력 깊이는 합성곱으로 계산할 필터의 수이며 32로 시작해서 64로 끝남

In [ ]:
# 컨브넷 위에 분류기 추가하기

# 완전연결층
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
# 출력층
model.add(layers.Dense(10, activation='softmax'))  # 10가지 숫자로 분류

In [ ]:
model.summary()

In [ ]:
# MNIST 이미지에 컨브넷 훈련하기
# feature 벡터화
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255 #0과 1사이의 값을 가지는 float값으로 변환

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

### 2장의 완전 연결 네트워크는 97% acc가 나옴. 더 향상된 결과
### 컨브넷이 더 간단한데 더 잘 작동하는 이유?: Conv2D, MaxPooling2D층 때문

5.1.1 합성곱 연산
- 완전연결층과 합성곱층 사이의 근본적 차이: 전자는 입력 특성 공간에 있는 전역패턴을 학습하나 후자는 지역패턴을 학습함. 이미지일 경우 2D 윈도우로 입력에서 패턴을 찾음
- 컨브넷의 성질
  - 평행 이동 불변성(translation invariant): 완전연결 네트워크는 새로운 위치에 나타난것은 새로운 패턴으로 학습해야 함. 한편 컨브넷은 이미지 특정 위치에서 패턴을 학습했을 경우 다른 곳에서도 패턴 인식이 가능함. 적은 수의 훈련 샘플을 사용하여 일반화 능력을 가진 표현을 학습 가능함. 이미지를 효율적으로 처리 가능(인간이 보는 세상도 평행 이동으로 인해 사물이 다르게 인식되지 않음)
  - 공간적 계층 구조: 첫번째 합성곱 층이 에지 같은 작은 지역 패턴을 학습하고, 두번쨰 합성곱 층은 첫번째 층의 특성으로 구성된 더 큰 패턴을 학습하는등 계층적으로 학습하는 방식. 복잡하고 추상적인 시각적 개념을 효과적으로 학습할 수 있음(인간이 보는 세상도 공간적 계층 구조를 가짐)
- 합성곱 연산은 특성맵(feature map)이라고 부르는 3D텐서에 적용됨. 이 텐서는 2개의 공간축(높이, 너비)과 깊이축(채널)로 구성됨. 합성곱 연산은 입력 특성맵에서 작은 패치(patch)들을 추출하고, 이런 모든 패치에 같은 변환을 적용하여 출력 특성맵(output feature map)을 만듦
- 출력 특성맵도 3D 텐서임. 깊이는 층의 매개변수로 결정되기에 상황에 따라 다름. 깊이축(채널)은 RGB 컬러 처럼 특정 컬러를 의미하지 않으며, 일종의 필터를 의미함. 필터는 입력 데이터의 어떤 특성을 인코딩 함(예: 입력값에 얼굴이 있는지)
- MNIST 예제에서 첫번째 합성곱 층이 (28,28,1) 크기의 특성맵을 입력으로 받아 (26,26,32) 크기의 특성맵을 출력함. 즉 입력에 대해 32개의 필터를 적용


## 5.2 소규모 데이터셋에서 밑바닥부터 컨브넷 훈련하기
- https://github.com/gilbutITbook/006975/blob/master/5.2-using-convnets-with-small-datasets.ipynb

5.2.1 작은 데이터셋 문제에서 딥러닝의 타당성

5.2.2 데이터 내려받기

In [ ]:
!pip uninstall keras
!pip uninstall tensorflow

In [ ]:
!pip install keras==2.3.1
!pip install tensorflow==2.2.0

In [ ]:
import os, shutil

In [ ]:
# 원본 데이터셋을 압축 해제한 디렉터리 경로
original_dataset_dir = '/content/drive/MyDrive/cakd3_colab/dl_keras/datasets/cats_and_dogs/train'

# 소규모 데이터셋을 저장할 디렉터리
base_dir = '/content/drive/MyDrive/cakd3_colab/dl_keras/datasets/cats_and_dogs_small'
if os.path.exists(base_dir):  # 반복적인 실행을 위해 디렉토리를 삭제합니다.
    shutil.rmtree(base_dir)   # 이 코드는 책에 포함되어 있지 않습니다.
os.mkdir(base_dir)

# 훈련, 검증, 테스트 분할을 위한 디렉터리
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# 훈련용 고양이 사진 디렉터리
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

# 훈련용 강아지 사진 디렉터리
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

# 검증용 고양이 사진 디렉터리
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

# 검증용 강아지 사진 디렉터리
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

# 테스트용 고양이 사진 디렉터리
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

# 테스트용 강아지 사진 디렉터리
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

# 처음 1,000개의 고양이 이미지를 train_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

# 다음 500개 고양이 이미지를 validation_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# 다음 500개 고양이 이미지를 test_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# 처음 1,000개의 강아지 이미지를 train_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# 다음 500개 강아지 이미지를 validation_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# 다음 500개 강아지 이미지를 test_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

In [ ]:
print('훈련용 고양이 이미지 전체 개수:', len(os.listdir(train_cats_dir)))
print('훈련용 강아지 이미지 전체 개수:', len(os.listdir(train_dogs_dir)))
print('검증용 고양이 이미지 전체 개수:', len(os.listdir(validation_cats_dir)))
print('검증용 강아지 이미지 전체 개수:', len(os.listdir(validation_dogs_dir)))
print('테스트용 고양이 이미지 전체 개수:', len(os.listdir(test_cats_dir)))
print('테스트용 강아지 이미지 전체 개수:', len(os.listdir(test_dogs_dir)))

5.2.3 네트워크 구성하기

In [ ]:
# 강아지 vs 고양이 분류를 위한 소규모 컨브넷 만들기
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
# 모델의 훈련 설정하기
from keras import optimizers

model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4),metrics=['acc'])

5.2.4 데이터 전처리
- 데이터는 네트워크에 주입되기 전 부동 소수 타입의 텐서로 전처리 되어 있어야 함
1. 사진 파일을 읽음
2. jpeg 콘텐츠를 rgb픽셀 값으로 디코딩
3. 부동 소수 타입의 텐서로 변환
4. 픽셀값(0~255)의 스케일을 [0,1]로 조정(신경망은 작은 입력값을 선호함)
- 케라스는 이 단계를 자동으로 처리하는 유틸리티가 있음(keras.preprocessing.image). 케라스의 ImageDataGenerator 클래스는 디스크에 있는 이미지 파일을 전처리된 배치 텐서로 자동으로 바꾸어주는 파이썬 제너레이터를 만들어 줌

In [ ]:
#  ImageDataGenerator를 사용하여 디렉터리에서 이미지 읽기
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=20, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size=(150,150), batch_size=20, class_mode='binary')

In [ ]:
# 배치 제너레이터를 사용하여 모델 훈련하기
history = model.fit_generator # fit_generator: fit과 동일하나 데이터 제너레이터 사용 가능
(train_generator, steps_per_epoch=100, epochs=30,  # steps_per_epoch: 횟수만큼 경사 하강법 단계를 실행
                              validation_data=validation_generator, validation_steps=50) # validation_steps: 검증 데이터 중 제너레이터에서 얼마나 많은 배치를 추출하여 평가할지

In [ ]:
# 모델 저장하기
model.save('cats_and_dogs_small_1.h5')

In [ ]:
# 훈련의 정확도와 손실 그래프 그리기
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.figure()

plt.plot(epochs,loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()

plt.show()

5.2.5 데이터 증식 사용하기

In [ ]:
# ImageDataGenerator를 사용하여 데이터 증식 설정하기
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1,
                             shear_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest')

In [ ]:
# 랜덤하게 증식된 훈련 이미지 그리기

from keras.preprocessing import image
fnames = sorted([os.path.join(train_cats_dir, fname) for fname in os.listdir(train_cats_dir)])
img_path = fnames[3]
img = image.load_img(img_path, target_size=(150,150))

x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0
for batch in datagen.flow(x, batch_size=1):
  plt.figure(i)
  imgplot = plt.imshow(image.array_to_img(batch[0]))
  i += 1
  if i % 4 == 0:
    break
    
plt.show()

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

# 검증 데이터는 증식되어서는 안 됩니다!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # 타깃 디렉터리
        train_dir,
        # 모든 이미지를 150 × 150 크기로 바꿉니다
        target_size=(150, 150),
        batch_size=32,
        # binary_crossentropy 손실을 사용하기 때문에 이진 레이블을 만들어야 합니다
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50)

In [ ]:
model.save('cats_and_dogs_small_2.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show() 

## 5.3 사전 훈련된 컨브넷 사용하기
- pretrained network: 대규모 이미지 분류 문제를 위해 대량의 데이터셋에서 미리 훈련되어 저장된 네트워크. 원본 데이터셋이 충분히 크고 일반적이라면 사전 훈련된 네트워크에 의해 학습된 특성의 계층 구조는 실제 세상에 대한 일반적인 모델로 효율적 역할 할 수 있음.
- 새로운 문제가 원래 작업과 완전히 다른 클래스에 대한 것이라도 이런 특성은 많은 컴퓨터 비전 문제에 유용함.(예: 동물이나 생활용품으로 이루어진 ImageNet 데이터셋에 네트워크 훈련후, 이 네트워크를 이미지에서 가구 아이템을 식별하는 것 같은 다른 용도로 사용할 수 있음)
- VGG16: 2014년 개발. ImageNet 데이터셋에 널리 사용되는 컨브넷 구조. 조금 오래되어 최고 성능은 못미치며 타 모델보다 무거우나, 모델 구조가 이전에 보았던것과 비슷함
  - VGG 16, ResNet, Inception, Inception-ResNet, Xception 등으로도 불림
  - 합성곱층 13개, 완전연결층 3개(총 16개). VGG19는 합성곱층 16개, 완전연결층 3개(총 19개)
  - pretrained network를 사용하는 방법: Feature Extraction(특성추출), Fine Tuning(미세조정)


5.3.1 특성 추출

1. 데이터 증식을 사용하지 않는 빠른 특성 추출
  - 새로운 데이터셋에서 합성곱 기반 층을 실행하고, 출력을 넘파이 배열로 디스크에 저장. 데이터를 독립된 완전 연결 분류기에 입력으로 사용함.
  - 합성곱 연산은 전체 과정 중 가장 비싼 부분임. 모든 입력 이미지에 대해 합성곱 기반층을 한번만 실행하면 되기에 빠르고 비용이 적게 듦. 이 이유로 이 기법에는 데이터 증식을 사용할 수 없음

2. 준비한 모델(conv_base)위에 Dense층을 쌓아 확장
  - 그 다음 입력 데이터에서 엔드-투-엔드로 전체 모델을 실행함.
  - 모델에 노출된 모든 입력 이미지가 매번 합성곱 기반층을 통과하기 때문에, 데이터 증식 사용 가능함. 첫번째 방식보다 비용이 많이 듦

In [ ]:
# VGG16 합성곱 기반 층 만들기

from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))

# 1. weight: 모델을 초기화할 가중치 체크포인트 지정
# 2. include_top: 네트워크의 최상위 완전 연결 분류기 포함 여부. 기본값은 True이며, ImageNet의 클래스 1000개에 대응되는 완전 연결 분류기를 포함함.
### 강아지와 고양이를 2개의 클래스로 구분하는 완전 연결층을 추가하려 하므로 최상위 완전 연결 분류기 포함 안함(False)
# 3. input_shape: 네트워크에 주입할 이미지 텐서의 크기. 옵션이며 지정 안할지 어떤 크기의 입력도 처리 가능함
### include_top=True일시 합성곱 층 위에 완전 연결층이 포함되므로 input_shape는 원본 모델과 동일한 (224,224,3)이 되어야 함

In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [ ]:
# 1. 데이터 증식을 사용하지 않는 빠른 특성 추출
# 사전 훈련된 합성곱 기반 층(conv_base)을 사용한 특성 추출하기

import os 
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = '/content/drive/MyDrive/cakd3_colab/dl_keras/datasets/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size=20

def extract_features(directory, sample_count):
  features = np.zeros(shape=(sample_count, 4,4,512))
  labels = np.zeros(shape=(sample_count))
  generator = datagen.flow_from_directory(directory, target_size=(150,150), batch_size=batch_size, class_mode='binary')

  i =0
  for inputs_batch, labels_batch in generator:
    features_batch = conv_base.predict(inputs_batch)
    features[i*batch_size: (i+1)*batch_size] = features_batch
    labels[i*batch_size: (i+1)*batch_size] = labels_batch
    i += 1
    if i * batch_size >= sample_count:
      break                             # 제너레이터는 루프 안에서 무한하게 데이터를 만들어내므로, 모든 이미지를 한번씩 처리하고 나면 중지시켜야함
  return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
# 추출된 특성의 크기는 (samples, 4,4,512)임. 완전 연결 분류기에 주입하기 위해 2D 사이즈 (samples, 8192)로 펼치기
train_features = np.reshape(train_features, (2000, 4*4*512))
validation_features = np.reshape(validation_features, (1000*4*4*512))
test_features = np.reshape(test_features, (1000*4*4*512))

In [ ]:
# 완전 연결 분류기를 정의하고 훈련하기

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers 

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4*4*512))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=optimizers.RMSprop(lr=2e-5), loss='binary_crossentropy', metrics=['acc'])
history = model.fit(train_features, train_labels, epochs=30, batch_size=20, validation_data=(validation_features, validation_labels))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/30
 99/100 [============================>.] - ETA: 0s - loss: 0.5167 - acc: 0.7621

ValueError: ignored

In [ ]:
# 결과 그래프 그리기

import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

## 5.4 컨브넷 학습 시각화

## 5.5 요약